# 2. 수작업 한 침수 및 비침수 (지리 동 )데이터와 강수 데이터 병합
 - #### <strong>2.1 busan_flood_base에서 지리데이터와 동 데이터를 수작업으로 작업 한 busan_flood_geo데이터를 가져와서 사용.</strong>

여기서 이제 동 및 코드까지 결합했지만 불완전했던 busan_flood_base에서 <br>
지리데이터와 동 데이터를 수작업으로 작업 한 busan_flood_geo데이터를 가져와서 사용한다.<br>

In [25]:
import pandas as pd

In [26]:
busan_flood_geo = pd.read_csv('../data/processing_data/1st/busan_flood_geo.csv')

# 필요한 컬럼 추출 및 컬럼명 수정

In [27]:
busan_flood_geo = busan_flood_geo.drop(columns=['FLUD_CAUSE', 'F_DSSTR_NM', 'F_CUE_DTL',
       'WRITNG_YM', 'FLUD_DPWT', 'DMG_AMOUNT', 
        'FLUD_WAL', 'CITY',  ])

In [28]:
busan_flood_geo.rename(columns={"FLUD_DPWT.1":"FLUD_DPWT","F_DSTRC_NM":"DISTRICT"},inplace=True)

In [29]:
time_rain = pd.read_csv('../data/processing_data/1st/busan_rain_data_mod.csv')

# SEQ 기준으로 inner 병합

In [30]:
BF_TR = pd.merge(busan_flood_geo, time_rain, how='inner', on = 'SEQ')

In [31]:
BF_TR = BF_TR.drop(columns='ADM_CD_y')
BF_TR.rename(columns={'ADM_CD_x' : 'ADM_CD'}, inplace=True)

# 시간당 최대 강우일 때만 침수심 남기고 모두 0으로 변경

In [32]:
SEQ_list = busan_flood_geo['SEQ'].unique()
for seq in SEQ_list:
    BF_TR.loc[(BF_TR["SEQ"]==seq) & (BF_TR["HOUR_RAINFALL"]!=BF_TR[BF_TR["SEQ"]==seq]['HOUR_RAINFALL'].max()),"FLUD_DPWT"]=0

In [33]:
BF_TR.to_csv('../data/processing_data/2nd/busan_flood_geo_rain.csv',index=False, encoding='utf-8')

In [34]:
busan_unflood_geo = pd.read_csv('../data/processing_data/1st/busan_Unflood_geo.csv')

In [35]:
busan_unflood_geo = pd.merge(busan_unflood_geo,time_rain, how='inner', on='SEQ')

In [36]:
busan_unflood_geo = busan_unflood_geo.drop(columns='ADM_CD_y')
busan_unflood_geo.rename(columns={'ADM_CD_x':'ADM_CD'},inplace=True)

In [37]:
busan_unflood_geo.to_csv('../data/processing_data/2nd/busan_Unflood_geo_rain.csv',index=False,encoding='utf-8')

# 가중치 함수

In [38]:
def slope_weight(slope_info):
    weight_data = slope_info.copy()
    for idx,col in enumerate(slope_info):
        if col <= 15:
            weight_data[idx] = col / (46)
        elif col > 15 and col <= 35:
            weight_data[idx] = col / (18 * 2**2)
        elif col > 35 and col <= 60:
            weight_data[idx] = col / (17 * 3**2)
        elif col > 60 and col <= 80:
            weight_data[idx] = col / (11 * 4**2)
        elif col > 80:
            weight_data[idx] = col / (6.28 * 5**2)

In [39]:
def high_weight(high_info):
    weight_data = high_info.copy()
    for idx,col in enumerate(high_info):
        if col <= 20:
            weight_data[idx] = col / (79.62 / 1**2)
        elif col > 20 and col <= 40:
            weight_data[idx] = col / (10.18 / 2**2)
        elif col > 40 and col <= 60:
            weight_data[idx] = col / (4.63 / 3**2)
        elif col > 60 and col <= 80:
            weight_data[idx] = col / (2.78 / 4**2)
        elif col > 80:
            weight_data[idx] = col / (2.78 / 5**2)
    return weight_data

In [40]:
def rain_weight(rain_info):
    weight_data = rain_info.copy()
    for idx,col in enumerate(rain_info):
        if col <= 15:
            weight_data[idx] = col / (46)
        elif col > 15 and col <= 35:
            weight_data[idx] = col / (18 * 2**2)
        elif col > 35 and col <= 60:
            weight_data[idx] = col / (17 * 3**2)
        elif col > 60 and col <= 80:
            weight_data[idx] = col / (11 * 4**2)
        elif col > 80:
            weight_data[idx] = col / (6.28 * 5**2)
    return weight_data

In [41]:
BF_TR['Hourly_Rainfall_Weight'] = rain_weight(BF_TR['HOUR_RAINFALL'])
BF_TR['HIGH_Weight'] = high_weight(BF_TR['HIGH'])
BF_TR['SLOPE_AVG_Weight'] = slope_weight(BF_TR['SLOPE_AVG'])

busan_unflood_geo['Hourly_Rainfall_Weight'] = rain_weight(busan_unflood_geo['HOUR_RAINFALL'])
busan_unflood_geo['HIGH_Weight'] = high_weight(busan_unflood_geo['HIGH'])
busan_unflood_geo['SLOPE_AVG_Weight'] = slope_weight(busan_unflood_geo['SLOPE_AVG'])

In [42]:
BF_TR.to_csv('../data/processing_data/2nd/busan_flood_weight.csv',index=False, encoding='utf-8')
busan_unflood_geo.to_csv('../data/processing_data/2nd/busan_Unflood_weight.csv',index=False,encoding='utf-8')